In [ ]:
from os import getcwd
import pandas as pd
from pathlib import Path
import numpy as np

In [ ]:
files = list(Path(getcwd().replace('notebooks', ''), 'test_data', 'wb').glob('*.xlsx'))
for f in files: print(f)

In [ ]:
dfs = []
for f in files:
    df = pd.read_excel(f, engine='calamine', date_format='YYYY-MM-DD')
    dfs.append(df)
df = pd.concat(dfs, ignore_index=True)
df = df.replace(np.nan, '')
df

In [ ]:

df = df[df['Название'] != np.nan]
sale_groups = df[df['Обоснование для оплаты']
                 == 'Продажа'].groupby('Название')
logistics_groups = df[df['Обоснование для оплаты'] == 'Логистика'].groupby('Название')
keeping_groups = df[df['Обоснование для оплаты'] == 'Хранение'].groupby('Название')
second_sale_groups = df[df['Тип документа'] == 'Продажа'].groupby('Название')
result = []
for name, grp in sale_groups:
    article = grp['Артикул поставщика'].iloc[0] if grp['Артикул поставщика'].iloc[0] != 0 else ''
    result.append({
        'Наименование товара': grp['Название'].iloc[0],
        'Артикул': grp['Артикул поставщика'].iloc[0],
        'Размер': grp['Размер'].iloc[0],
        'Количество продаж': grp['Кол-во'].sum(),
        'Средняя цена продажи': round(grp['Цена розничная'].mean(), 2),
        'Комиссия МП': round(grp['Цена розничная'].mean(), 2) - round(grp['К перечислению Продавцу за реализованный Товар'].mean(), 2),
        'Логистика': logistics_groups['Услуги по доставке товара покупателю'].sum()[name] / grp['Кол-во'].sum()
    })

In [ ]:
pd.DataFrame(result)